In [8]:
%matplotlib inline
import os
import numpy as np
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from scipy.sparse import csr_matrix
import networkx as nx

from scipy.linalg import orthogonal_procrustes

import torch
import torch.optim as optim
from torch_geometric.data import Data

from sklearn.manifold import TSNE
from sklearn.datasets import make_moons  # 导入make_moons函数

from util import Net, GIN, GAT, stationary, reconstruct, dG, GCN
from torch.nn import MSELoss


np.random.seed(0)
torch.manual_seed(0)

n = 2000
m = 500
# DISTANCE = 0.1

# 导入moon数据集
x, y = make_moons(n_samples=n, noise=0.1, random_state=0) 
# 分割训练集和测试机
n_train = int(n * 0.7)
train_ind = torch.randperm(n)[:n_train]
test_ind = torch.LongTensor(list(set(np.arange(n)) - set(train_ind.tolist())))

print(f"Train size: {n_train}\nTest size: {n-n_train}")



Train size: 1400
Test size: 600


In [9]:
from torchmetrics import Accuracy

accuracy = Accuracy(task='binary')

def get_accs(function: str, distance):
    D = pairwise_distances(x)  # Assuming you have defined pairwise_distances elsewhere
    A_binary = np.where(D <= distance, 1, 0)
    row_indices, col_indices = np.where(A_binary == 1)
    edges = list(zip(row_indices, col_indices))

    # Convert to tensors
    x_tensor = torch.tensor(x, dtype=torch.float)
    y_tensor = torch.tensor(y, dtype=torch.long)
    edges_tensor = torch.tensor(edges, dtype=torch.long).t().contiguous()
    data = Data(x=x_tensor, y=y_tensor, edge_index=edges_tensor)

    function = eval(function)
    net = function(m)
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    net.train()

    for epoch in range(100):
        ind = torch.eye(n)[:, torch.randperm(n)[:m]]
        X_extended = torch.hstack([x_tensor, ind])
        data = Data(x=X_extended, edge_index=edges_tensor)
        rec = net(data)
        loss = dG(x_tensor[train_ind], rec[train_ind])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Predict labels for the test set
    net.eval()
    with torch.no_grad():
        rec_test = net(data)
        pred_labels = torch.argmax(rec_test[test_ind], dim=1)
    # Calculate accuracy
    acc = accuracy(pred_labels, y_tensor[test_ind])
    return acc


In [10]:
# Import necessary libraries
import json

# Define epsilon range
eps_range = [eps for eps in np.arange(0.01, 1, 0.05)]

# Initialize dictionaries to store accuracy results
accs_GCN = {}
accs_GIN = {}
accs_GAT = {}

# Iterate over epsilon values
for eps in eps_range:
    accs_GCN[eps] = get_accs('GCN', eps)
    accs_GIN[eps] = get_accs('GIN', eps)
    accs_GAT[eps] = get_accs('GAT', eps)

# Write results to text files
with open('./results/accs_GCN.txt', 'w') as file:
    file.write(json.dumps(accs_GCN))

with open('./results/accs_GIN.txt', 'w') as file:
    file.write(json.dumps(accs_GIN))

with open('./results/accs_GAT.txt', 'w') as file:
    file.write(json.dumps(accs_GAT))


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import json

# Load accuracy data from text files
with open('./results/accs_GCN.txt', 'r') as file:
    accs_GCN = json.load(file)

with open('./results/accs_GIN.txt', 'r') as file:
    accs_GIN = json.load(file)

with open('./results/accs_GAT.txt', 'r') as file:
    accs_GAT = json.load(file)

# Extract epsilon values and corresponding accuracies
eps_values = list(accs_GCN.keys())
accs_values_GCN = list(accs_GCN.values())
accs_values_GIN = list(accs_GIN.values())
accs_values_GAT = list(accs_GAT.values())

# Plot the accuracies
plt.plot(eps_values, accs_values_GCN, label='GCN')

# Add labels and title
plt.xlabel('Epsilon')
plt.ylabel('Accuracy')
plt.title('Accuracy of GCN eps blob')

# Add legend
plt.legend()

# Show plot
plt.grid(True)
plt.savefig("./results/eps_blob_GCN.jpg")
plt.show()


In [ ]:
# Plot the accuracies
plt.plot(eps_values, accs_values_GIN, label='GIN')

# Add labels and title
plt.xlabel('Epsilon')
plt.ylabel('Accuracy')
plt.title('Accuracy of GIN eps blob')

# Add legend
plt.legend()

# Show plot
plt.grid(True)
plt.savefig("./results/eps_blob_GIN.jpg")
plt.show()


In [ ]:
# Plot the accuracies
plt.plot(eps_values, accs_values_GAT, label='GAT')

# Add labels and title
plt.xlabel('Epsilon')
plt.ylabel('Accuracy')
plt.title('Accuracy of GAT eps blob')

# Add legend
plt.legend()

# Show plot
plt.grid(True)
plt.savefig("./results/eps_blob_GAT.jpg")
plt.show()
